# Predicting Energy Consumption with XGBoost

Regression analysis can be used to model the relationship between a dependent variable and one or more independent variables. 
- In the context of time series analysis, regression can be used to model the relationship between a time series variable and one or more predictor variables.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

Lets read and show the data

In [ ]:
df = pd.read_csv('data/PJME_hourly.csv')
df = df.set_index("Datetime")
df.head()

In [ ]:
df.plot(figsize=(8,4), style='.')
plt.show()

In [ ]:
df.index

In [ ]:
df.index = pd.to_datetime(df.index)
df.plot(figsize=(8,4), style='.')
plt.show()

In [ ]:
df[(df.index >= '2006-01-01') & (df.index <= '2006-12-31')].plot()

In [ ]:
df[(df.index >= '2006-01-01') & (df.index <= '2006-01-31')].plot()

In [ ]:
df.loc[(df.index > '01-01-2010') & (df.index < '01-08-2010')] \
    .plot(figsize=(15, 5), title='Week Of Data')
plt.show()

## Train / Test Split

Unlike traditional train/test splitting, where the data is randomly divided into training and testing sets, in time series analysis, the data is split in chronological order, such that the training set includes data from earlier time periods and the testing set includes data from later time periods.

In [ ]:
train = df.loc[df.index < '01-01-2015']
test = df.loc[df.index >= '01-01-2015']

fig, ax = plt.subplots(figsize=(15, 5))
train.plot(ax=ax, label='Training Set', title='Data Train/Test Split')
test.plot(ax=ax, label='Test Set')
ax.axvline('01-01-2015', color='black', ls='--')
ax.legend(['Training Set', 'Test Set'])
plt.show()

Now, lets create some features that are relevant to the forecasting, and that are related to the date

In [ ]:
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df

df = create_features(df)
df.head()

### Visualize our Feature / Target Relationship

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
sns.boxplot(data=df, x='hour', y='PJME_MW')
ax.set_title('MW by Hour')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.boxplot(data=df, x='month', y='PJME_MW', palette='Blues')
ax.set_title('MW by Month')
plt.show()

## Create our Model

In [ ]:
train = create_features(train)
test = create_features(test)

FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year']
TARGET = 'PJME_MW'

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

We will use XGB regressor, a very powerfull classifier based on decision trees and boosting

In [ ]:
reg = xgb.XGBRegressor(n_estimators=1000, early_stopping_rounds=50)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],        
        verbose=True)

You can see that the error in the validation set starts to increase, so we have overfit the model.

Lets try with a small learning rate to move slower ..

In [ ]:
reg = xgb.XGBRegressor(n_estimators=1000, 
                       early_stopping_rounds=50, learning_rate=0.001)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],        
        verbose=100)

Lets move now a little faster in order to find a better solution

In [ ]:
reg = xgb.XGBRegressor(n_estimators=1000, 
                       early_stopping_rounds=50, learning_rate=0.01)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],        
        verbose=100)

## Feature Importance

In [ ]:
reg.feature_importances_

Values are meaningless, so lets improve the visualization by creating a new dataframe

In [ ]:
fi = pd.DataFrame(data=reg.feature_importances_,
             index=reg.feature_names_in_,
             columns=['importance'])
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.show()

Lets see the correlations between features ...

In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.1f')

## Forecast on Test

In [ ]:
y_pred = reg.predict(X_test)
y_pred

For a better visualization, lets add the predicted values to the original dataset

In [ ]:
test['prediction'] = y_pred
df = df.merge(test[['prediction']], how='left', left_index=True, right_index=True)
ax = df[['PJME_MW']].plot(figsize=(15, 5))
df['prediction'].plot(ax=ax, style='.')
plt.legend(['Truth Data', 'Predictions'])
ax.set_title('Raw Dat and Prediction')
plt.show()

In a week ..

In [ ]:
ax = df.loc[(df.index > '04-01-2018') & (df.index < '04-08-2018')]['PJME_MW'] \
    .plot(figsize=(15, 5), title='Week Of Data')
df.loc[(df.index > '04-01-2018') & (df.index < '04-08-2018')]['prediction'] \
    .plot(style='.')
plt.legend(['Truth Data','Prediction'])
plt.show()

Results are not as good as they can be, but the regressor have managed to capture the seasonality.

## Score (RMSE)

In [ ]:
score = np.sqrt(mean_squared_error(test['PJME_MW'], test['prediction']))
print(f'RMSE Score on Test set: {score:0.2f}')

Look at the worst predicted days ... where they hollidays?

In [ ]:
test['error'] = np.abs(test[TARGET] - test['prediction'])
test['date'] = test.index.date
test.groupby(['date'])['error'].mean().sort_values(ascending=False).head(10)

In [ ]:
test.error.plot()
ax.set_title('Errors per day')
plt.show()

Lets zoom the errors in a year

In [ ]:
test.error.loc[(test.index > '01-01-2017') & (test.index < '01-01-2018')].plot()
ax.set_title('Errors per day in a year')
plt.show()

And zoom in a week

In [ ]:
test.error.loc[(test.index > '04-01-2018') & (test.index < '04-08-2018')].plot()
ax.set_title('Errors per day in a week')
plt.show()

Plenty of room to improve:
- More robust cross validation
- Add more features (weather forecast, holidays)

## Outlier Analysis and removal

In [ ]:
df['PJME_MW'].plot(kind='hist', bins=500)

Lets first consider those points with largest values

In [ ]:
df.query('PJME_MW > 50_000')['PJME_MW'] \
    .plot(style='.',
          figsize=(15, 5),
          color=color_pal[5],
          title='Outliers')

They are consistent with the times of the year where the demand is higher.

Now, lets see the days with lowest consumption

In [ ]:
df.query('PJME_MW < 20_000')['PJME_MW'] \
    .plot(style='.',
          figsize=(15, 5),
          color=color_pal[5],
          title='Outliers')

Most of the points are consistent, but some of them have a nasty behavior. 
- Maybe they are real values, but not usefull for predicting in the future.

In [ ]:
df.query('PJME_MW < 19_000')['PJME_MW'] \
    .plot(style='.',
          figsize=(15, 5),
          color=color_pal[5],
          title='Outliers')

In [ ]:
df = df.query('PJME_MW > 19_000').copy()

## Time Series Cross Validation

The traditional k-fold cross-validation approach is not suitable for time series data because it assumes that the data is independently and identically distributed, which is not true for time series data. 
- Instead, we need to use a modified form of cross-validation that takes into account the temporal dependencies in the data.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits=5, test_size=24*365*1, gap=0)
df = df.sort_index()

In [ ]:
tss.split(df)

It is generator ...

In [ ]:
iterator = iter(tss.split(df))
train_range, test_range = next(iterator)
print(train_range)
print(test_range)

In [ ]:
train_range, test_range = next(iterator)
print(train_range)
print(test_range)

In [ ]:
train_range, test_range = next(iterator)
print(train_range)
print(test_range)

It is clearer in a graphical representation

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(15, 15), sharex=True)

fold = 0
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]
    train['PJME_MW'].plot(ax=axs[fold],
                          label='Training Set',
                          title=f'Data Train/Test Split Fold {fold}')
    test['PJME_MW'].plot(ax=axs[fold],
                         label='Test Set')
    axs[fold].axvline(test.index.min(), color='black', ls='--')
    fold += 1
plt.show()

## Forecasting Horizon

The forecast horizon is the length of time into the future for which forecasts are to be prepared. 
- These generally vary from short-term forecasting horizons (less than three months) to long-term horizons (more than two years).

## Lag Features

In time series analysis, lag features are variables that represent past values of the time series variable being analyzed. 
- Lag features are used to capture the autocorrelation or dependency of the time series variable on its past values.

In [ ]:
def add_lags(df):
    target_map = df['PJME_MW'].to_dict()
    df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
    df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
    return df

add_lags(df)

In [ ]:
df[df.index > '2010-01-01'][['PJME_MW','lag1', 'lag2', 'lag3']].head()

## Using Lags and Cross Validation together

In [ ]:
tss = TimeSeriesSplit(n_splits=5, test_size=24*365*1, gap=24)
df = df.sort_index()


fold = 0
preds = []
scores = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    train = create_features(train)
    test = create_features(test)

    FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month','year',
                'lag1','lag2','lag3']
    TARGET = 'PJME_MW'

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

In [ ]:
scores

In [ ]:
np.mean(scores)

Lets see the importance of the lag features for the last regressor

In [ ]:
fi = pd.DataFrame(data=reg.feature_importances_,
             index=reg.feature_names_in_,
             columns=['importance'])
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.show()

In [ ]:
test['prediction'] = y_pred
test.PJME_MW.plot()
test.prediction.plot()

In [ ]:
test[(test.index > '04-01-2018') & (test.index < '04-08-2018')].PJME_MW.plot()
test[(test.index > '04-01-2018') & (test.index < '04-08-2018')].prediction.plot()
plt.show()

## Predicting the Future

- Retraining on all data
- To Predict the future we need an emtpy dataframe for future date ranges.
- Run those dates through our feature creation code + lag creation